**Author:** [Fatemeh Chajaei](https://github.com/FatemehCh97)

# Load Packages

In [ ]:
!pip install keras_unet_collection
from keras_unet_collection import losses
import keras

In [ ]:
import os
import glob
import numpy as np
from tqdm import notebook, tnrange
import matplotlib.pyplot as plt
import random
from osgeo import gdal

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img, save_img
from keras.models import load_model

# Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Amsterdam
pred_dir = "/content/drive/My Drive/Amsterdam Data"

In [ ]:
# Set some parameters
im_width = 512
im_height = 512

In [ ]:
pred_img_id = [] # list of names all images in the given path
for f in glob.glob(os.path.join(pred_dir, "*.tif")):
    pred_img_id.append(os.path.split(f)[1].split(".")[0])
pred_img_id.sort()
len(pred_img_id)

In [ ]:
X_pred = np.zeros((len(pred_img_id), im_height, im_width, 1), dtype=np.float32)
# X_pred = np.zeros((len(pred_img_id), im_height, im_width, 3), dtype=np.float32) # DeepLabV3+

In [ ]:
for n, id_ in notebook.tqdm(enumerate(pred_img_id), total=len(pred_img_id)):
    # Load images
    pred_img = load_img(os.path.join(pred_dir, id_+".tif"), color_mode = "grayscale")
    pred_x_img = img_to_array(pred_img)
    # Normalization
    X_pred[n] = (pred_x_img - pred_x_img.min()) / (pred_x_img.max() - pred_x_img.min())

In [ ]:
np.shape(X_pred)

# Plot Samples

In [ ]:
# visualize samples and their predicted label
def plot_sample(X, preds, binary_preds, ix=None):

    if ix is None:
        ix = random.randint(0, len(X))

    fig, ax = plt.subplots(1, 2, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='gray')
    ax[0].set_title('DSM')


    ax[1].imshow(binary_preds[ix].squeeze(), cmap='gray', vmin=0, vmax=1)
    ax[1].set_title('Predicted Building Footprint')


# Prediction

In [ ]:
with tf.device('/CPU:0'):
    model = load_model("/content/drive/MyDrive/MegaImport/unet3p_pool_unpool.h5", compile=False)

In [ ]:
# unet3+ loss/compile
def hybrid_loss(y_true, y_pred):

    loss_focal = losses.focal_tversky(y_true, y_pred, alpha=0.5, gamma=4/3)
    loss_iou = losses.iou_seg(y_true, y_pred)

    # (x)
    #loss_ssim = losses.ms_ssim(y_true, y_pred, max_val=1.0, filter_size=4)

    return loss_focal+loss_iou #+loss_ssim

model.compile(loss=[hybrid_loss, hybrid_loss, hybrid_loss, hybrid_loss, hybrid_loss],
                          loss_weights=[0.25, 0.25, 0.25, 0.25, 1.0],
                          optimizer=keras.optimizers.Adam(learning_rate=1e-4))

In [ ]:
keras.models.Model.summary(model)

In [ ]:
preds = model.predict(X_pred, batch_size=4, verbose=1)

In [ ]:
preds_unet3p = preds[-1]
preds_tresh = (preds_unet3p > 0.5).astype(np.uint8)

In [ ]:
plot_sample(X_pred, y_test, preds_unet3p, preds_tresh, ix=None)

## Save

In [ ]:
save_path = "/content/unet3p_amst_pred"

In [ ]:
gt = []
for n, id_ in notebook.tqdm(enumerate(pred_img_id), total=len(pred_img_id)):
    # Load images
    pred_img_t = gdal.Open(pred_dir+"/"+id_+".tif")
    gt.append(pred_img_t.GetGeoTransform())

In [ ]:
for i, image in enumerate(preds_tresh, 0):
    geotransform = (gt[i][0], gt[i][1], gt[i][2], gt[i][3], gt[i][4], gt[i][5])
    img_raster = gdal.GetDriverByName('GTiff').Create(os.path.join(save_path, f'Pred_{i}.tif'), 512, 512, 1, gdal.GDT_Float32)
    image_reshape = image.reshape(512, -1)
    img_raster.GetRasterBand(1).WriteArray(image_reshape)
    img_raster.SetGeoTransform(geotransform)
    img_raster.SetProjection(pred_img_t.GetProjection())
    img_raster.FlushCache()

In [ ]:
!zip -r /content/unet3p_amst_pred.zip /content/unet3p_amst_pred